In [7]:
from langchain.chat_models import ChatOpenAI
import openai
from dotenv import load_dotenv
import os
from langchain.schema import SystemMessage, HumanMessage

load_dotenv()

True

In [9]:
openai_client = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hello")
]
response = openai_client(
    messages
)


response.content

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 18, 'total_tokens': 27, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-56facbc2-48bc-47b6-bbe3-4a2788b34959-0')